# Manual Experiments


## X Dataset

### Filter and import `X.json`

In [13]:

filter_dataset(input_file=X_JSON, pattern=RETWEET_PATTERN, inplace=True)


NameError: name 'filter_dataset' is not defined

In [ ]:
save_dataset_to_csv(input_file=X_JSON, output_file=X_CSV)


In [ ]:
x = pd.read_csv(filepath_or_buffer=X_CSV)
x


### Import Naïve Bayes model and Predict binary label

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts=x['body'])

gnb_dict = joblib.load(filename='models/optimal_gnb.pkl')
gnb = gnb_dict['model']


In [ ]:
x_embedding_matrix = embedding_for_vocab(
    filepath=GLOVE_50D_MODEL,
    word_index=tokenizer.word_index,
    embedding_dim=EMBEDDING_50D_DIM
)

x_embedding = embed_texts(
    texts=x['body'],
    tokenizer=tokenizer,
    embedding_matrix=x_embedding_matrix
)

x['predicted_labels'] = gnb.predict(X=x_embedding)


In [ ]:
counts = Counter(x['predicted_labels'])
print(dict(counts))


## Reddit Dataset

### Filter and import `X+Reddit.json`

In [ ]:
filter_dataset(input_file=X_REDDIT_JSON, pattern=RETWEET_PATTERN, inplace=True)


In [ ]:
save_dataset_to_csv(input_file=X_REDDIT_JSON, output_file=X_REDDIT_CSV)


In [ ]:
x_reddit = pd.read_csv(filepath_or_buffer=X_REDDIT_CSV)
x_reddit


### Import Naïve Bayes model and Predict binary label

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_reddit['body'])

gnb_dict = joblib.load('models/optimal_gnb.pkl')
gnb = gnb_dict['model']


In [ ]:
x_reddit_embedding_matrix = embedding_for_vocab(
    filepath=GLOVE_50D_MODEL,
    word_index=tokenizer.word_index,
    embedding_dim=EMBEDDING_50D_DIM
)

x_reddit_embedding = embed_texts(
    texts=x_reddit['body'],
    tokenizer=tokenizer,
    embedding_matrix=x_reddit_embedding_matrix
)

x_reddit['predicted_labels'] = gnb.predict(X=x_reddit_embedding)


In [ ]:
counts = Counter(x_reddit['predicted_labels'])
print(dict(counts))


In [ ]:
s_list = [
    'Quantum computers could imperil the security of confidential electronic information, such as emails. To counter this threat, NIST has finalized its set of three encryption algorithms designed to withstand a future quantum computer’s cyberattacks: https://nist.gov/news-events/news/2024/08/nist-releases-first-3-finalized-post-quantum-encryption-standards',
    'Adversaries can deliberately manipulate AI systems to make them malfunction — and there’s no foolproof defense. \n\nCheck out NIST’s new guide to the types of attacks developers and users can expect, along with approaches to mitigate them: https://nist.gov/news-events/news/2024/01/nist-identifies-types-cyberattacks-manipulate-behavior-ai-systems',
    'In my new blog for @_CPResearch_\n I propose a new injection technique, using the Thread Name API - check it out! 💙',
    ' Russian Hacker Extradited to US for Phobos Ransomware Charges ',
    ' Critical Windows Kerberos Flaw Exposes Millions of Servers to Attack - PATCH NOW ',
    'Phishing emails increasingly use SVG attachments to evade detection',
    'A cautionary tale from Graham Cluley at Tripwire about a New Hampshire man who has pleaded guilty to charges after having successfully tricked staff of his past employer – Motorola – to provide him with their login credentials to help him with a “task awaiting approval.” His phishing emails got them to click on a link to a site that asked them to provide this information. He also sent an SMS message to a Motorola employee asking for, and receiving, their MFA code. After his arrest he attempted to order a false passport, even writing to New Hampshire Senator Maggie Hassan asking that his application be expedited. That stunt might add 10 years to the potential 20 years for the Motorola hack.'
]


In [ ]:
predicted_labels = gnb.predict(X=embed_texts(
    texts=s_list,
    tokenizer=tokenizer,
    embedding_matrix=embedding_for_vocab(
        filepath=GLOVE_50D_MODEL,
        word_index=tokenizer.word_index,
        embedding_dim=EMBEDDING_50D_DIM
    )
))

predicted_labels


In [ ]:
# Example usage:
text = "I built this website (free, no ads or anything) and I am desperate for some feedback... :-)\n\nhttps://computerprogramming.art/"

# Testing the function
print(gnb.predict(X=embed_texts(
    texts=[process_text_with_links(text)],
    tokenizer=tokenizer,
    embedding_matrix=embedding_for_vocab(
        filepath=GLOVE_50D_MODEL,
        word_index=tokenizer.word_index,
        embedding_dim=EMBEDDING_50D_DIM
    )
)))

## Keyword Classification Anastasia Dataset

### Anastasia Preprocessing


In [ ]:
models = {}


In [ ]:
anastasia = pd.read_csv(filepath_or_buffer=ANASTASIA_CSV)
anastasia = anastasia[anastasia['lang'] == 'en']
anastasia.info()


In [ ]:
# Create a Rake instance
rake = Rake()

if Path.exists(ANASTASIA_KEYWORDS_CSV):
    anastasia_with_keywords = pd.read_csv(filepath_or_buffer=ANASTASIA_KEYWORDS_CSV)
else:
    anastasia_with_keywords = pd.DataFrame(columns=['text', 'target'])

    # Text from which keywords will be extracted
    for text, label in zip(anastasia['full_text'], anastasia['Related']):
        # Extract keywords from the text
        rake.extract_keywords_from_text(text=process_text_with_links(text))

        # Get the ranked keywords
        keywords = rake.get_ranked_phrases_with_scores()
        for _, keyword in keywords[:2]:
            temp = pd.DataFrame({'text': keyword, 'target': label}, index=[0])
            anastasia_with_keywords = pd.concat([anastasia_with_keywords, temp], ignore_index=True)

    anastasia_with_keywords.to_csv(path_or_buf=ANASTASIA_KEYWORDS_CSV, index=False)

anastasia_with_keywords.info()


### Training Naïve Bayes


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(anastasia_with_keywords['text'])

model_dict = gaussian_naive_bayes_trainer(
    model_path=GLOVE_50D_MODEL,
    tokenizer=tokenizer,
    embedding_dim=50,
    texts=anastasia_with_keywords['text'],
    target=anastasia_with_keywords['target'],
    cv=NUM_FOLDS,
    random_state=RANDOM_SEED,
    test_size=0.4
)

models['Naïve Bayes 50D'] = model_dict


In [ ]:
model_dict = gaussian_naive_bayes_trainer(
    model_path=GLOVE_100D_MODEL,
    tokenizer=tokenizer,
    embedding_dim=100,
    texts=anastasia_with_keywords['text'],
    target=anastasia_with_keywords['target'],
    cv=NUM_FOLDS,
    random_state=RANDOM_SEED,
    test_size=0.4
)

models['Naïve Bayes 100D'] = model_dict


In [ ]:
model_dict = gaussian_naive_bayes_trainer(
    model_path=GLOVE_200D_MODEL,
    tokenizer=tokenizer,
    embedding_dim=200,
    texts=anastasia_with_keywords['text'],
    target=anastasia_with_keywords['target'],
    cv=NUM_FOLDS,
    random_state=RANDOM_SEED,
    test_size=0.4
)

models['Naïve Bayes 200D'] = model_dict


In [ ]:
model_dict = gaussian_naive_bayes_trainer(
    model_path=GLOVE_300D_MODEL,
    tokenizer=tokenizer,
    embedding_dim=300,
    texts=anastasia_with_keywords['text'],
    target=anastasia_with_keywords['target'],
    cv=NUM_FOLDS,
    random_state=RANDOM_SEED,
    test_size=0.4
)

models['Naïve Bayes 300D'] = model_dict


### Evaluation


In [ ]:
plt.figure(figsize=(9, 6))

for name, model_dict in models.items():
    plt.plot(
        model_dict['fpr'], model_dict['tpr'],
        label=f"{name}: {model_dict['roc_auc']:.2f}"
    )

plt.title('ROC-AUC Curve')
plt.plot([0, 1], [0, 1], 'k--', label='Random Guess: 0.50')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
max_auc = 0.
optimal_gnb = ''

for name, model in models.items():
    if models[name]['roc_auc'] > max_auc:
        max_auc = models[name]['roc_auc']
        optimal_gnb = model

joblib.dump(optimal_gnb, GNB_KEYWORDS_MODEL, compress=9)
del optimal_gnb

model = joblib.load(GNB_KEYWORDS_MODEL)
model


### Predict X Dataset

In [ ]:
# Create a Rake instance
rake = Rake()

if Path.exists(X_KEYWORDS_CSV):
    x_with_keywords = pd.read_csv(filepath_or_buffer=X_KEYWORDS_CSV)
else:
    x_with_keywords = pd.DataFrame(columns=['text'])

    # Text from which keywords will be extracted
    for text in x['body']:
        # Extract keywords from the text
        rake.extract_keywords_from_text(text=process_text_with_links(text))

        # Get the ranked keywords
        keywords = rake.get_ranked_phrases_with_scores()
        for _, keyword in keywords[:2]:
            temp = pd.DataFrame({'text': keyword}, index=[0])
            x_with_keywords = pd.concat([x_with_keywords, temp], ignore_index=True)

    x_with_keywords.to_csv(path_or_buf=X_KEYWORDS_CSV, index=False)

x_with_keywords.info()


In [ ]:
x = pd.read_csv(filepath_or_buffer=X_CSV)
x = x[['title', 'body']]

tokenizer = Tokenizer()

gnb = model['model']

x_embedding_matrix = embedding_for_vocab(
    filepath=GLOVE_50D_MODEL,
    word_index=tokenizer.word_index,
    embedding_dim=EMBEDDING_50D_DIM
)

x_embedding = embed_texts(
    texts=x['body'],
    tokenizer=tokenizer,
    embedding_matrix=x_embedding_matrix
)

x['predicted_labels'] = gnb.predict(X=x_embedding)

counts = Counter(x['predicted_labels'])
print(dict(counts))


In [ ]:
x['body'].apply(func=process_text_with_links)
x


## Experimenting with Custom NER

In [ ]:
anastasia_df = pd.read_csv(filepath_or_buffer='data/anastasia.csv')
anastasia_df


In [ ]:
nlp = spacy.load("en_core_web_trf")
sentence = anastasia_df['full_text'][0]
doc = nlp(sentence)

for token in doc.ents:
    print(f'Text:\t{token.text}\nStart Position:\t{token.start_char}\nLabel:\t{token.label_}\n')
